In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
from tensorflow import keras

In [4]:
tf.random.set_seed(42)

In [5]:
import tensorflow_datasets as tfds

dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [6]:
info.splits

{'train': <SplitInfo num_examples=3670, num_shards=2>}

In [7]:
info.splits["train"]

<SplitInfo num_examples=3670, num_shards=2>

In [8]:
class_names = info.features["label"].names
class_names

['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses']

In [9]:
n_classes = info.features["label"].num_classes

In [10]:
dataset_size = info.splits["train"].num_examples
dataset_size

3670

In [11]:
test_set_raw, valid_set_raw, train_set_raw = tfds.load(
    "tf_flowers",
    split=["train[:10%]", "train[10%:25%]", "train[25%:]"],
    as_supervised=True)

In [12]:
def preprocess(image, label):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

In [13]:
def central_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]])
    top_crop = (shape[0] - min_dim) // 4
    bottom_crop = shape[0] - top_crop
    left_crop = (shape[1] - min_dim) // 4
    right_crop = shape[1] - left_crop
    return image[top_crop:bottom_crop, left_crop:right_crop]

def random_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]]) * 90 // 100
    return tf.image.random_crop(image, [min_dim, min_dim, 3])

def preprocess(image, label, randomize=False):
    if randomize:
        cropped_image = random_crop(image)
        cropped_image = tf.image.random_flip_left_right(cropped_image)
    else:
        cropped_image = central_crop(image)
    resized_image = tf.image.resize(cropped_image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

In [14]:
from functools import partial

In [15]:
batch_size = 32
train_set = train_set_raw.shuffle(1000).repeat()
train_set = train_set.map(partial(preprocess, randomize=True)).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)

In [16]:
mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

In [17]:
with mirrored_strategy.scope():
    base_model = keras.applications.xception.Xception(weights="imagenet",
                                                  include_top=False)
    avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
    output = keras.layers.Dense(n_classes, activation="softmax")(avg)
    model = keras.models.Model(inputs=base_model.input, outputs=output)
    for layer in base_model.layers:
        layer.trainable = False
    optimizer = keras.optimizers.SGD(learning_rate=0.2, momentum=0.9,weight_decay=0.01)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
                  metrics=["accuracy"])
    history = model.fit(train_set,
                        steps_per_epoch=int(0.75 * dataset_size / batch_size),
                        validation_data=valid_set,
                        validation_steps=int(0.15 * dataset_size / batch_size),
                        epochs=5)

83683744/83683744 [==============================] - 0s 0us/step
Epoch 1/5
86/86 [==============================] - 28s 129ms/step - loss: 1.5910 - accuracy: 0.7823 - val_loss: 2.5420 - val_accuracy: 0.8088
Epoch 2/5
86/86 [==============================] - 9s 106ms/step - loss: 1.1483 - accuracy: 0.8779 - val_loss: 1.2678 - val_accuracy: 0.8346
Epoch 3/5
86/86 [==============================] - 8s 93ms/step - loss: 0.6686 - accuracy: 0.8921 - val_loss: 1.3582 - val_accuracy: 0.8290
Epoch 4/5
86/86 [==============================] - 8s 96ms/step - loss: 0.5247 - accuracy: 0.9081 - val_loss: 1.0576 - val_accuracy: 0.8603
Epoch 5/5
86/86 [==============================] - 8s 92ms/step - loss: 0.4947 - accuracy: 0.9004 - val_loss: 1.1260 - val_accuracy: 0.8474


In [18]:
with mirrored_strategy.scope():
    for layer in base_model.layers:
        layer.trainable = True

    optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9,
                                     nesterov=True, weight_decay=0.001)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
                  metrics=["accuracy"])
    history = model.fit(train_set,
                        steps_per_epoch=int(0.75 * dataset_size / batch_size),
                        validation_data=valid_set,
                        validation_steps=int(0.15 * dataset_size / batch_size),
                        epochs=40)

Epoch 1/40
86/86 [==============================] - 70s 337ms/step - loss: 0.4328 - accuracy: 0.8681 - val_loss: 0.6680 - val_accuracy: 0.8566
Epoch 2/40
86/86 [==============================] - 27s 315ms/step - loss: 0.1699 - accuracy: 0.9411 - val_loss: 0.2515 - val_accuracy: 0.9154
Epoch 3/40
86/86 [==============================] - 27s 315ms/step - loss: 0.1089 - accuracy: 0.9644 - val_loss: 0.2830 - val_accuracy: 0.9062
Epoch 4/40
86/86 [==============================] - 28s 322ms/step - loss: 0.0804 - accuracy: 0.9735 - val_loss: 0.2596 - val_accuracy: 0.9173
Epoch 5/40
86/86 [==============================] - 27s 317ms/step - loss: 0.0753 - accuracy: 0.9749 - val_loss: 0.2703 - val_accuracy: 0.9191
Epoch 6/40
86/86 [==============================] - 27s 315ms/step - loss: 0.0416 - accuracy: 0.9869 - val_loss: 0.2432 - val_accuracy: 0.9265
Epoch 7/40
86/86 [==============================] - 27s 318ms/step - loss: 0.0471 - accuracy: 0.9887 - val_loss: 0.2675 - val_accuracy: 0.9301